<a href="https://colab.research.google.com/github/summii/data-science-profile/blob/master/Autoencoders.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install imageio

In [0]:
from tensorflow.keras.layers import Input, Dense, Conv2D, MaxPooling2D, UpSampling2D
from tensorflow.keras.models import Model
from keras import backend as K
import os
import glob
import imageio
import numpy as np
import cv2
from keras import regularizers
from keras.datasets import mnist


In [15]:
!git clone https://github.com/zaidalyafeai/QuickDraw10

fatal: destination path 'QuickDraw10' already exists and is not an empty directory.


In [0]:
import numpy as np

train_data = np.load('QuickDraw10/dataset/train-ubyte.npz')
test_data = np.load('QuickDraw10/dataset/test-ubyte.npz')

x_train, y_train = train_data['a'], test_data['b']

x_test, y_test = test_data['a'], test_data['b']

In [17]:
x_train = np.expand_dims(x_train.astype('float32') / 255., 3)
x_test = np.expand_dims(x_test.astype('float32') / 255., 3)

print(x_train.shape)
print(x_test.shape)

(80000, 28, 28, 1)
(20000, 28, 28, 1)


In [0]:
def create_layers():
  layers = []
  size = 32 
  
  #encoder layers
  for i in range(0, 3):
    x = Conv2D(size, (3, 3), activation='relu', padding='same')
    layers += [x] 
    x = MaxPooling2D((2, 2), padding='same')
    layers += [x]
    size = size // 2
  

  #deocder layers 
  for i in range(0, 3):
    size = size * 2
    if i == 2:
      x = Conv2D(size, (3, 3), activation='relu')
    else:
      x = Conv2D(size, (3, 3), activation='relu', padding='same')
    layers += [x]
    x = UpSampling2D((2, 2))
    layers += [x]
    
    
  x = Conv2D(1, (3, 3), activation='sigmoid', padding='same')
  layers += [x]
  
  return layers

def autoencoder():
  input_img = Input(shape=(28, 28, 1))  

  layers = create_layers()

  #create the auto encoder network 
  x = input_img
  for layer in layers:
    x = layer(x)
    
  autoencoder = Model(input_img, x)
  autoencoder.compile(optimizer = 'adam', loss = 'binary_crossentropy')
  
  #create the encoder network
  x = input_img
  for layer in layers[0:6]:
    x = layer(x)
    
  encoder = Model(input_img, x)
  
  #create the decoder network
  input_encoded = Input(shape = (4, 4, 8))
  x = input_encoded
  for layer in layers[6:]:
    x = layer(x)

  decoder = Model(input_encoded, x)
  return autoencoder, encoder, decoder

In [0]:
autoencoder, encoder, decoder = autoencoder()

In [21]:
autoencoder.fit(x_train, x_train,
                epochs=100,
                batch_size=128,
                shuffle=True,
                validation_data=(x_test, x_test))

Train on 80000 samples, validate on 20000 samples
Epoch 1/100
80000/80000 [==============================] - 18s 219us/sample - loss: 0.3006 - val_loss: 0.2462
Epoch 2/100
80000/80000 [==============================] - 4s 46us/sample - loss: 0.2328 - val_loss: 0.2239
Epoch 3/100
80000/80000 [==============================] - 4s 46us/sample - loss: 0.2191 - val_loss: 0.2146
Epoch 4/100
80000/80000 [==============================] - 4s 46us/sample - loss: 0.2117 - val_loss: 0.2089
Epoch 5/100
80000/80000 [==============================] - 4s 45us/sample - loss: 0.2066 - val_loss: 0.2048
Epoch 6/100
80000/80000 [==============================] - 4s 46us/sample - loss: 0.2031 - val_loss: 0.2018
Epoch 7/100
80000/80000 [==============================] - 4s 45us/sample - loss: 0.2005 - val_loss: 0.1995
Epoch 8/100
80000/80000 [==============================] - 4s 46us/sample - loss: 0.1982 - val_loss: 0.1973
Epoch 9/100
80000/80000 [==============================] - 4s 46us/sample - loss: 0.